# Proyecto: Informe de Emails Recibidos


## Introducción

### Problema a abordar

La problemática radica en la gran cantidad de mails que recibimos todos los días. En esa marea de mails, se suelen pasar por alto mails importantes. Muchas veces, un mail queda perdido en el buzón de entrada y luego se hace difícil encontrarlo. Si además se refería a algo urgente que necesitaba verse a tiempo, se torna un problema mayor.

### Desarrollo de la propuesta de solución

La propuesta es generar un informe de envío semanal a una casilla de correo que haga un repaso de todos los mails recibidos en la última semana, de manera que nos permita ponernos al día con los asuntos a atender. Se buscará que el informe ordene los temas relevados según orden de prioridad.<br>
La idea consiste en acceder a un mail en Outlook o exportar los mails de la última semana y a través de un prompt leer los mails en cuestión y armar un resumen de estos ordenándolos por prioridad.<br>
Para usar también el modelo texto a imagen, buscaremos que el informe comience con un banner generado por este modelo.



**Prompts a utilizar:**

- **Texto a Imagen:**

```text

"Full-bleed dark gradient analytics banner (blue to purple). "
"Glowing charts: donut on left, vertical bars center-left, orange line chart across right. "
"Tiny HUD dots and thin grid lines. Vector, crisp, modern UI, high contrast. "
"Centered composition with safe padding top and bottom; avoid elements at edges. No text."

```

- **Texto a Texto:**

```text

ROLE
You analyze recent emails and output ONE single artifact: a JSON email draft whose BODY IS the weekly report.

METHOD (tight)
1) Leer cuidadosamente cada email (asunto, remitente, cuerpo). Extraer datos clave (vencimientos, montos, cuentas, períodos, tickets, links).
2) Determinar la acción necesaria (pagar, responder, adjuntar, confirmar, coordinar, monitorear).
3) Organizar y priorizar: primero vencidos/vence hoy; luego pendientes de respuesta; luego seguimiento/colaboración; listar todos los NO LEÍDOS.
4) Diseñar un reporte claro y accionable, resaltando insights/patrones (sin suposiciones). Usar íconos y color (HTML) con equivalencia en texto plano.
5) Formatear el reporte para envío por email (body_text y body_html equivalentes).
6) Devolver el JSON final **y nada más**, cumpliendo el schema.

...

```


### Viabilidad del proyecto
El proyecto lo podemos dividir en 3 etapas:

#### 1) Obtener mails

Primero necesitamos tener el contenido de los correos electrónicos recibidos en una casilla de correo. Para esto, se buscará acceder a la bandeja de entrada de Outlook o automatizar la exportación de los mails de la última semana para luego poder entregarle el archivo a Gemini.

#### 2) Generar informe

La segunda etapa sería cuando entra en juego el uso de los modelos a través de las API. Como modelo de texto a texto, se usará Gemini para analizar todos los mails y armar el informe. Como modelo de texto a imagen, se buscará en Hugging Face alguno gratuito para crear el banner. En esta etapa, se necesitará pulir el prompt en ambos modelos para lograr el objetivo.

#### 3) Enviar informe

La tercera y última etapa consiste en enviar por email el informe generado.

Las 3 etapas mencionadas son altamente viables de ser llevadas a cabo con las herramientas a disposición.

## Objetivos

Sistematizar la lectura de correos para transformar información dispersa en un informe ejecutivo claro y accionable.

Estandarizar criterios de priorización y seguimiento para mejorar la toma de decisiones.

## Metodología

El flujo implementado debe convertir un alto volumen de correos en un producto informativo breve, verificable y anonimizado, apto para lectura ejecutiva y para integrar a tableros/automatizaciones.

Las 3 etapas mencionadas se llevan a cabo de la siguiente manera:

**Etapa 1: Obtener mails (Recolección y depuración)**

- Obtengo correos desde Microsoft Graph (Outlook) del período de análisis y se eliminan duplicidades, ruidos y datos superfluos, preservando la trazabilidad.
- Normalizo campos (remitente, asunto, cuerpo, fechas, flags/“Destacado”, adjuntos, webLink). Excluyo correos de promociones, newsletters y enviados por mí salvo que sean relevantes (respuestas a clientes).

**Etapa 2: Generar informe (Diseño de prompt y orquestación)**

- Uso instrucciones de sistema claras (rol, estilo, longitud máxima, idioma) y secciones obligatorias (urgentes, pendientes, seguimiento y no leídos) para producir un resumen ejecutivo con secciones fijas de menos de 200 palabras.
- Aplico anonimización para reemplazar emails y nombres propios por placeholders para poder subirlo a un repositorio publico de Github. 


**Etapa 3: Enviar Informe**

- Genero informe resumen según el formato buscado y lo envío en un mail a la misma casilla de correo de donde se obtuvieron los correos relevados.

## Herramientas y tecnologías

Se utilizan:
- Visual Studio Code
- Jupyter Notebook
- Github

- Lenguaje: **Python**
- Modelo AI: **Google Gemini**, **Flux.1 Schnell** 

**Tecnica de Prompt:**

Se busca utilizar en primera instancia la tecnica zero-shot para no incrementar el alto consumo de tokens que se prevee. La cantidad de tokens va a ser alta de por sí por la cantidad de correos que tiene que analizar el modelo. Sumado a eso, en funcion de obtener el resultado deseado, el prompt va a requerir un mayor detalle y, por ende, una cantidad importante de tokens. Aunque se trata de optimizar el consumo de tokens, se evaluará si el resultado obtenido es acorde a lo esperado y, en caso de no serlo, se optará por agregar ejemplos (few-shot) para obtener respuestas breves y bien formateadas.

## Implementación

A continuación se incluye el código para llegar a la solución propuesta:


In [ ]:
# ===== Generacion de banner : uso de modelo texto a imagen =====
import os
from huggingface_hub import InferenceClient
from dotenv import load_dotenv
from PIL import Image, ImageFilter
from pathlib import Path

ASSETS = Path("assets"); ASSETS.mkdir(exist_ok=True)
BANNER_PATH = ASSETS / "banner.png"   # ruta fija


# Tamaños
FINAL_W, FINAL_H = 1400, 120      # exactamente el espacio del reporte
GEN_W, GEN_H     = 1024, 240      # más alto para mejor detalle (no se recorta)

# Modelos: probamos otro y dejamos sdxl-turbo como fallback
MODELS = [
    "black-forest-labs/FLUX.1-schnell",   # rápido, buenos banners
    "stabilityai/sdxl-turbo",             # fallback rápido
]

# Prompt: estilo similar al ejemplo, sin texto/etiquetas/axes
PROMPT = (
    "Full-bleed dark gradient analytics banner (blue to purple). "
    "Glowing charts: donut on left, vertical bars center-left, orange line chart across right. "
    "Tiny HUD dots and thin grid lines. Vector, crisp, modern UI, high contrast. "
    "Centered composition with safe padding top and bottom; avoid elements at edges. No text."
)
NEGATIVE = (
    "text, words, letters, numbers, digits, labels, captions, watermark, logo, axis, axes, "
    "ticks, scale marks, frame, border, people, hands, blurry, lowres, artifacts"
)

STEPS    = 9           # 8–12 va bien
GUIDANCE = 0.0
SEED     = 1337        # cambiá para otra variación

def make_client(model_id: str, token: str) -> InferenceClient:
    # Usamos provider de la Hosted Inference API para evitar el 'auto'
    return InferenceClient(model=model_id, provider="hf-inference", api_key=token)

def generate(model_id: str, token: str) -> Image.Image:
    client = make_client(model_id, token)
    return client.text_to_image(
        prompt=PROMPT,
        negative_prompt=NEGATIVE,
        width=GEN_W,
        height=GEN_H,
        num_inference_steps=STEPS,
        guidance_scale=GUIDANCE,
        seed=SEED,
    )

def main():
    load_dotenv()
    token = os.getenv("HUGGINGFACE_API_KEY")
    assert token, "Falta HUGGINGFACE_API_KEY en .env"

    img = None
    last_err = None
    for mid in MODELS:
        try:
            img = generate(mid, token)
            print(f"✅ Generado con: {mid}")
            break
        except Exception as e:
            print(f"⚠️ Falló {mid}: {e!r}")
            last_err = e

    if img is None:
        raise RuntimeError(f"No se pudo generar ninguna imagen. Último error: {last_err!r}")

    # Solo reescalar (sin recortar) al tamaño exacto del banner
    banner = img.resize((FINAL_W, FINAL_H), Image.LANCZOS)
    banner = banner.filter(ImageFilter.UnsharpMask(radius=0.5, percent=70, threshold=2))

    banner.save(BANNER_PATH, format="PNG")

if __name__ == "__main__":
    main()

✅ Generado con: black-forest-labs/FLUX.1-schnell


In [28]:
# ===== Generacion del informe: uso del modelo de texto a texto =====

import os, json, re
from datetime import datetime, timedelta
from dotenv import load_dotenv
from google import genai
from google.genai import types  # <- para GenerateContentConfig
from pathlib import Path

# Fecha con zona horaria AR
try:
    from zoneinfo import ZoneInfo  # Python 3.9+
except Exception:
    ZoneInfo = None


# =========================
# Config
# =========================
load_dotenv()

#_MODEL_ENV = os.getenv("GOOGLE_GEMINI_MODEL", "gemini-2.5-flash")
_MODEL_ENV = os.getenv("GOOGLE_GEMINI_MODEL", "gemini-2.5-pro")
MODEL = _MODEL_ENV.replace("models/", "")
# El cliente toma GEMINI_API_KEY o GOOGLE_API_KEY desde el entorno
client = genai.Client()

# mails a enviar copia
EMAIL_REPORT_CC= ""
EMAIL_REPORT_BCC=""

# banner path
ASSETS_DIR = Path("assets")
BANNER_PATH = ASSETS_DIR / "banner.png"

# === Config única (editar solo acá) ===
CONFIG = {
    "from_mailreader": True,
    "limit": 1000, #limite de mails a buscar
    "days_back": 30, #cantidad de dias hacia atras para buscar mails
    "json_path": None,
    "max_chars": 120_000, # maxima cantidad de caracteres a mandar al modelo
    "out": None,                 # "respuesta.txt" o None
    "model": MODEL,              # o "gemini-2.5-pro", etc.
    "temperature": None,         # ej. 0,3
    "max_output_tokens": None,   # ej. 1200
    "response_mime_type": "application/json",  # ej. "text/markdown"
    "debug": False,
}

# =========================
# Response schema (EMAIL ONLY)
# =========================
EMAIL_ONLY_SCHEMA = types.Schema(
  type=types.Type.OBJECT,
  properties={
    "email_draft": types.Schema(
      type=types.Type.OBJECT,
      properties={
        "body_text":types.Schema(type=types.Type.STRING),
        "body_html":types.Schema(type=types.Type.STRING),
      },
      required=["body_text","body_html"]
    )
  },
  required=["email_draft"]
)


# =========================
# Helpers
# =========================

def load_rows_from_json(path: str) -> tuple[list[dict], str | None]:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    if isinstance(data, dict) and "rows" in data:
        rows = data["rows"]
    elif isinstance(data, list):
        rows = data
    else:
        raise ValueError("El JSON no es una lista de correos.")
    return rows, None

def load_rows_from_mailreader(limit: int | None = None, days_back: int | None = None) -> tuple[list[dict], str]:
    """
    Lee correos con mail_reader y, en la misma función, obtiene la casilla actual desde Graph /me.
    Devuelve (rows, my_email). Si no hay casilla, falla.
    """
    try:
        import mail_reader as mr
    except Exception as e:
        raise RuntimeError(f"No pude importar mail_reader: {e}")

    # === 1) Leer correos ===
    if hasattr(mr, "collect_rows"):
        try:
            rows = mr.collect_rows(limit=limit, days_back=days_back or None, debug=False)
        except TypeError:
            rows = mr.collect_rows(limit=limit)
    elif hasattr(mr, "main"):
        out_path = os.getenv("OUTPUT_JSON", "emails.json")
        mr.main()
        rows = load_rows_from_json(out_path)[0]  # nuestra load_rows_from_json ahora devuelve (rows, None)
    else:
        raise RuntimeError("mailreader no expone collect_rows() ni main().")

    # === 2) Resolver casilla via Graph /me (mismo token/caché del mailreader) ===
    client_id = getattr(mr, "CLIENT_ID", None) or os.getenv("CLIENT_ID")
    tenant_id = getattr(mr, "TENANT_ID", None) or os.getenv("TENANT_ID", "common")
    if not client_id:
        raise RuntimeError("CLIENT_ID no definido en mail_reader ni en variables de entorno.")

    app, persist_cache = mr.build_msal_app(client_id, tenant_id)
    token = mr.acquire_token(app, persist_cache)
    access_token = token.get("access_token")
    if not access_token:
        raise RuntimeError("No se obtuvo access_token para Microsoft Graph.")

    me = mr.graph_get(
        f"{mr.GRAPH}/me",
        access_token,
        params={"$select": "mail,userPrincipalName,otherMails"}
    )

    my_email = None
    for k in ("mail", "userPrincipalName"):
        v = (me.get(k) or "").strip()
        if "@" in v:
            my_email = v
            break
    if not my_email:
        for v in (me.get("otherMails") or []):
            v = (v or "").strip()
            if "@" in v:
                my_email = v
                break

    # Fallback final: helper del módulo si existe
    if not my_email and hasattr(mr, "get_my_addresses"):
        try:
            addrs = mr.get_my_addresses(access_token) or []
            for v in addrs:
                if isinstance(v, str) and "@" in v:
                    my_email = v.strip()
                    break
        except Exception:
            pass

    if not my_email:
        raise RuntimeError("No se pudo determinar la casilla desde Graph /me (mail/userPrincipalName/otherMails).")

    return rows, my_email

## Banner
def _banner_block() -> str:
    """Devuelve el <img> inline (base64) si existe assets/banner.png, o '' si no existe."""
    try:
        if BANNER_PATH.exists():
            import base64
            b64 = base64.b64encode(BANNER_PATH.read_bytes()).decode()
            return (
                f'<img src="data:image/png;base64,{b64}" '
                f'style="display:block;width:100%;max-width:900px;height:auto;'
                f'border-radius:12px;margin:0 0 16px 0;">'
            )
    except Exception:
        pass
    return ""


# === Body reducer / compactor ===
# para reducir la cantidad de tokens

_SIG_PAT = re.compile(
    r"(?:^--\s*$|^Enviado desde|^Sent from|^On .+ wrote:|^El .+ escribió:)",
    re.IGNORECASE | re.MULTILINE
)
_QUOTE_PAT = re.compile(r"^\s*>.*$", re.MULTILINE)  # líneas citadas tipo '>'
_URL_PAT = re.compile(r"https?://\S+")
_MULTI_NL = re.compile(r"\n{3,}")

def clean_body(txt: str, limit: int = 400) -> str:
    if not txt:
        return ""
    # 1) eliminar líneas citadas y URLs
    txt = _QUOTE_PAT.sub("", txt)
    txt = _URL_PAT.sub("", txt)
    # 2) cortar en firma/respuesta previa si aparece
    m = _SIG_PAT.search(txt)
    if m:
        txt = txt[:m.start()]
    # 3) colapsar saltos extra y espacios
    txt = _MULTI_NL.sub("\n\n", txt).strip()
    # 4) trunc final
    return txt[:limit]


def clamp_rows(rows: list[dict], max_chars: int = 120_000) -> list[dict]:
    """Devuelve filas *compactas* y recorta por caracteres de forma consistente."""
    compact_rows, total = [], 0
    for r in rows:
        compact = {
            "Fecha": r.get("Fecha"),
            "ReceivedUTC": r.get("ReceivedUTC"),
            "Remitente": r.get("Remitente"),
            "RemitenteNombre": r.get("RemitenteNombre"),
            "Asunto": r.get("Asunto"),
            #"Cuerpo": (r.get("Cuerpo") or "")[:1000],  # límite duro por mail
            "Cuerpo": clean_body(r.get("Cuerpo") or "", limit=400),  # límite duro por mail
            "TieneAdjuntos": bool(r.get("TieneAdjuntos")),
            "Leido": bool(r.get("Leido")),
            "Respondido": bool(r.get("Respondido")),
            "Categorias": r.get("Categorias"),
            "WebLink": r.get("WebLink"),
        }
        #s = json.dumps(compact, ensure_ascii=False)
        s = json.dumps(compact, ensure_ascii=False, separators=(',', ':'))
        
        if total + len(s) > max_chars and compact_rows:
            break
        compact_rows.append(compact)
        total += len(s)
    return compact_rows


# =========================
# Utils: emails & JSON robusto
# =========================

_EMAIL_RE = re.compile(r"[A-Za-z0-9._%+\-]+@[A-Za-z0-9.\-]+\.[A-Za-z]{2,}")

def _split_emails(raw: str) -> list[str]:
    if not raw:
        return []
    # Split por coma, punto y coma, espacios y saltos de línea
    parts = re.split(r"[,\s;]+", raw.strip())
    return [p for p in parts if p]

def _filter_valid_emails(parts: list[str]) -> list[str]:
    return [p for p in parts if _EMAIL_RE.fullmatch(p or "")]

def _dedupe_preserve_order(items: list[str]) -> list[str]:
    seen = set()
    out = []
    for it in items:
        k = it.lower()
        if k not in seen:
            seen.add(k)
            out.append(it)
    return out

def get_env_cc_bcc() -> tuple[list[str], list[str]]:
    cc_raw  = EMAIL_REPORT_CC
    bcc_raw = EMAIL_REPORT_BCC

    cc  = _dedupe_preserve_order(_filter_valid_emails(_split_emails(cc_raw)))
    bcc = _dedupe_preserve_order(_filter_valid_emails(_split_emails(bcc_raw)))
    return cc, bcc

def extract_json_payload(s: str) -> str:
    """Intenta extraer el primer bloque JSON { ... } (quita cercos ``` si aparecen)."""
    if not s:
        return s
    txt = s.strip()
    # quitar fences ``` o ```json
    if txt.startswith("```"):
        txt = re.sub(r"^```(?:json)?\s*", "", txt)
        txt = re.sub(r"\s*```$", "", txt)
    # recortar al primer {...} completo
    start = txt.find("{")
    end   = txt.rfind("}")
    if start != -1 and end != -1 and end > start:
        return txt[start:end+1]
    return txt

_URL_IN_HTML_RE = re.compile(r'(?<!href=["\'])https?://[^\s<>"\']+')

def urls_to_anchor_html(html: str) -> str:
    html = html or ""
    return _URL_IN_HTML_RE.sub(lambda m: f'<a href="{m.group(0)}">Ver mail</a>', html)

def urls_to_label_text(txt: str) -> str:
    txt = txt or ""
    return re.sub(r'https?://\S+', 'Ver mail', txt)




# =========================
# System instruction (email-only)
# =========================
def build_system_instruction(date_from: str) -> str:
    return f"""
Title: Weekly Email Report

ROLE
You analyze recent emails and output ONE single artifact: a JSON email draft whose BODY IS the weekly report.

METHOD (tight)
1) Leer cuidadosamente cada email (asunto, remitente, cuerpo). Extraer datos clave (vencimientos, montos, cuentas, períodos, tickets, links).
2) Determinar la acción necesaria (pagar, responder, adjuntar, confirmar, coordinar, monitorear).
3) Organizar y priorizar: primero vencidos/vence hoy; luego pendientes de respuesta; luego seguimiento/colaboración; listar todos los NO LEÍDOS.
4) Diseñar un reporte claro y accionable, resaltando insights/patrones (sin suposiciones). Usar íconos y color (HTML) con equivalencia en texto plano.
5) Formatear el reporte para envío por email (body_text y body_html equivalentes).
6) Devolver el JSON final **y nada más**, cumpliendo el schema.

CONTRACT (STRICT)
1) Output = SOLO JSON (sin fences) que cumpla exactamente el schema (abajo).
2) Idioma: español, tono profesional y conciso.
3) Privacidad: reemplazar nombres por [REDACTED], emails por [MAIL] y nombres de empresas por [EMPRESA] en TODO donde aparezcan (asunto, remitente, líneas). 
4) Enlaces obligatorios: toda línea que refiera a un email específico DEBE incluir su webLink.
   - En body_text: mostrar **Ver mail** (no URL cruda).
   - En body_html: mostrar **<a href="...">Ver mail</a>** (no URL cruda).
5) Sin invenciones: si un dato no está, dejar el campo vacio. No inventar montos, fechas ni adjuntos.**Nunca excluir** un correo por faltar un dato secundario.
6) Deduplicación conservadora por hilo/tema (normalizar asunto quitando “RE:”/“FW:” y espacios):
   - **Solo** deduplicar si los correos no agregan **nueva acción/fecha/monto**.
   - Si hay varias acciones distintas en un mismo hilo, crear **líneas separadas**.
   - Excepción: en **No leídos** no agrupar nunca.
7) **Fechas**: formato **DD-MM-YYYY** en todo el cuerpo (incluida la primera línea y etiquetas de vencimiento).
8) **Prioridades (usar íconos y color)**
   - 🔴 Crítico (vencido) → etiqueta: “[atrasado N días (DD-MM-YYYY)]”
   - 🟠 Vence hoy → etiqueta: “[vence hoy DD-MM-YYYY]”
   - 🟡 Vence en ≤3 días → etiqueta: “[vence en ≤3 días DD-MM-YYYY]”
   - 🟢 Informativo/menor (seguimiento)
9) Cada línea puede incluir **nota** (8-25 palabras) y, si corresponde, **próximo paso** o **impacto** explícitos.
10) **Cobertura**: incluir **todos** los correos **no-newsletter**. Si el texto se hace largo, **mantener todos los ítems** y acortar la “nota”.

OUTPUT SCHEMA (email-only)
{{
  "email_draft": {{
    "body_text": "string",
    "body_html": "string"
  }}
}}

RESTRICCIÓN
- NO incluyas "to", "cc", "bcc" ni "subject".

BODY TEMPLATE (NO imprimas “Encabezado” literalmente)
- Línea de contexto (primera línea):
Resumen del correo electrónico recibido desde el día **{date_from}** hasta la fecha.

- Línea de conteo (una sola línea):
⏰ Urgencias: X · 🧩 Acciones pendientes: Y · 📨 No leídos: Z

- **Asuntos Urgentes y Vencimientos Críticos (Atención Inmediata)**
(orden: vencidos → hoy → ≤3 días; deduplicados por tema)
• 🔴 [atrasado N días (DD-MM-YYYY)] Asunto normalizado — **Acción:** … — nota: (8-25 palabras) - link: Ver mail (+N similares)
• 🟠 [vence hoy DD-MM-YYYY] Asunto normalizado — **Acción:** … — nota: … — link: Ver mail
• 🟡 [vence en ≤3 días DD-MM-YYYY] Asunto normalizado — **Acción:** … — nota: … — link: Ver mail

- **Acciones Pendientes**
(solicitudes explícitas que requieren tu respuesta/confirmación y aún no fueron respondidas)
• 🧩 Asunto normalizado — **Acción:** … — nota: (qué pide/qué falta) — link: Ver mail

- **Tareas de Seguimiento y Colaboración**
(temas en curso que no son urgencia ni pedido directo, no incluir newsletters)
• 🟢 Tópico/tema — **Próximo paso:** … — nota: (monto, banco, cuenta, ticket, período) — link: Ver mail

- **No leídos**
(listar absolutamente **todos** los correos con Leido == false; más recientes primero; SIN agrupar)
• 📨 Asunto normalizado — Remitente [REDACTED] — nota: (frase breve del cuerpo) — link: Ver mail
(Si el volumen excede el límite, mostrar los que entren y cerrar con “+N restantes”.)

CLASSIFICATION RULES
- Urgente/crítico: due_date <= hoy o cuerpo con “vencido”, “vence hoy”, “último aviso”, etc.
- Pendiente: pedidos directos (“¿podés…?”, “por favor enviá…”, “necesito…”) sin respuesta.
- Seguimiento/colaboración: temas en curso que no son urgencia ni pedido directo, pero requieren coordinación (reuniones, tickets, configuraciones, informes).
- No leídos: TODOS los Leido == false del dataset (sin agrupar).
- Si detectás varios emails del mismo tema, deduplicar con “(+N similares)” excepto en No leídos.
- Excluir unicamente **Newsletter/FYI**
- No repetir emails en las distintas secciones. Ubicar cada email en la seccion más apropiada. Si Leido = false, va en la seccion **No leídos**.


HTML REQUIREMENTS
- `body_html` equivalente a `body_text`; permitido: <div>, <p>, <strong>, <ul>, <li>, <a>, <span>, <hr>, <img>.
- Íconos Unicode (🔴 🟠 🟡 🟢 ⏰ 🧩 🤝 📨 ⚠️ 📊 📎) y color en <span style="...">.
- Incluir banner superior (placeholder):
  <div style="width:100%;height:120px;background:#0F172A;border-radius:8px;margin:0 0 16px 0;"></div>

VALIDATION
- Si una sección no puede completarse sin adivinar, omitirla por completo.
- Todos los nombres de personas de personas se deben reemplazar nombres por [REDACTED], los emails por [MAIL] y las empresas por [EMPRESA].
- Devolver SOLO el JSON con `email_draft`, sin texto extra.

"""


# =========================
# Build user contents
# =========================
def build_contents(rows: list[dict]) -> list[str]:
    """
    Returns a list[str] with the payload (SDK turns each into UserContent).
    We serialize each email row as a compact JSON line and split into blocks.
    """
    intro = "Below is the mailing list in compact JSON lines:"
    #Serializamos cada correo en una línea compacta
    lines = [json.dumps(r, ensure_ascii=False, separators=(',', ':')) for r in rows]
    #Partimos en bloques de 30 líneas para que no sea una sola cadena enorme 
    BATCH = 30
    blocks = ["\n".join(lines[i:i+BATCH]) for i in range(0, len(lines), BATCH)]
    return [intro] + blocks


# Robustly extract JSON from GEMINI response
def extract_json_from_response(resp) -> str:
    """
    Try multiple places where the JSON can live in the Google GenAI response.
    Falls back to '', never raises.
    """
    # 1) Fast path: aggregated text
    try:
        txt = getattr(resp, "text", None)
        if txt and txt.strip():
            return txt
    except Exception:
        pass

    # 2) Candidates/parts: text or inline_data (base64) with application/json
    try:
        cands = getattr(resp, "candidates", None) or []
        for c in cands:
            content = getattr(c, "content", None)
            parts = getattr(content, "parts", None) or []
            for p in parts:
                # Plain text
                if hasattr(p, "text") and p.text:
                    return p.text
                # Inline JSON blob
                inline = getattr(p, "inline_data", None)
                if inline and getattr(inline, "mime_type", "") == "application/json":
                    b64 = getattr(inline, "data", "") or ""
                    if b64:
                        import base64
                        try:
                            return base64.b64decode(b64).decode("utf-8", "ignore")
                        except Exception:
                            pass
    except Exception:
        pass

    return ""

# =========================
# Ejecutor
# =========================
def run_analysis(config: dict):
    """Ejecuta el análisis leyendo TODO de config."""
    from_mailreader     = config["from_mailreader"]
    limit               = config["limit"]
    days_back           = config["days_back"]
    json_path           = config["json_path"]
    max_chars           = config["max_chars"]
    out                 = config["out"]
    model               = (config["model"] or MODEL).replace("models/", "")
    temperature         = config["temperature"]
    max_output_tokens   = config["max_output_tokens"]
    response_mime_type  = config["response_mime_type"]
    debug               = config["debug"]

    # Zona horaria AR
    if ZoneInfo:
        now_ar = datetime.now(ZoneInfo("America/Argentina/Buenos_Aires"))
    else:
        now_ar = datetime.now()

    date_from = (now_ar - timedelta(days=days_back)).strftime("%d-%m-%Y")

    # 1) Carga de correos
    if from_mailreader:
        rows, my_email = load_rows_from_mailreader(limit=limit, days_back=days_back)
    else:
        jp = json_path or os.getenv("OUTPUT_JSON", "emails.json")
        rows, my_email = load_rows_from_json(jp)

    if not rows:
        return "", {"error": "No hay correos para analizar."}
    if from_mailreader and not my_email:
        return "", {"error": "No se pudo resolver la casilla desde mailreader/Graph."}

    # Log de información
    #print(f"Correos cargados: {len(rows)}")
    #print(f"Email del usuario: {my_email}")
    #print(f"Período: últimos {days_back} días desde {date_from}")
    

    # 2) Orden + recorte
    rows_sorted  = sorted(rows, key=lambda r: r.get("ReceivedUTC") or r.get("Fecha") or "", reverse=True)
    rows_compact = clamp_rows(rows_sorted, max_chars=max_chars)

    # 3) Prompt: System + contents
    system_instruction = build_system_instruction(date_from)
    contents = build_contents(rows_compact)

    # 3.bis) Conteo simple SIN usar system_instruction en config
    def _tok(res):
        return getattr(res, "total_tokens", None) or getattr(res, "total_tokens_count", 0)

    # tokens SOLO del system: lo contamos como si fuera el primer contenido
    t_system_only = _tok(client.models.count_tokens(
        model=model,
        contents=[system_instruction]
    ))

    # tokens TOTALES de entrada: system + contents reales (intro + JSON)
    t_input_total = _tok(client.models.count_tokens(
        model=model,
        contents=[system_instruction] + contents
    ))

    # tokens de correos ≈ total - system
    t_emails_est = max(t_input_total - t_system_only, 0)


    # 4) Llamada al modelo
    try:
        response = client.models.generate_content(
            model=model,
            contents=contents,
            config=types.GenerateContentConfig(
                system_instruction=system_instruction,
                response_mime_type=response_mime_type if response_mime_type else None,
                response_schema=EMAIL_ONLY_SCHEMA,
                max_output_tokens=max_output_tokens if max_output_tokens else None,
                temperature=temperature if temperature else None, 
            ),
        )
    except Exception as e:
        if debug:
            raise
        return "", {"error": f"Fallo generate_content: {e}", "model": model}
    
    # 4.bis) Extraer JSON de forma robusta
    raw_payload = extract_json_from_response(response)
    text_for_json = extract_json_payload(raw_payload)

    if not text_for_json or not text_for_json.strip():
        return "", {
            "error": "El modelo no devolvio JSON valido (payload vacio).",
            "model": model,
            "rows_after_clamp": len(rows_compact),
            "raw_excerpt": (raw_payload or "")[:600]
        }

    #raw_text = getattr(response, "text", "") or ""
    #text_for_json = extract_json_payload(raw_text)

    # 5) Post-proceso determinista: fijar destinatario, subject y CC/BCC
    try:
        data = json.loads(text_for_json)
        if not isinstance(data, dict) or "email_draft" not in data:
            raise ValueError("Respuesta inválida: falta 'email_draft'.")
        draft = data["email_draft"]
        if not isinstance(draft, dict):
            raise ValueError("'email_draft' no es objeto.")
        # Subject estándar (AR)
        if ZoneInfo:
            today_ar = datetime.now(ZoneInfo("America/Argentina/Buenos_Aires")).strftime("%d-%m-%Y")
        else:
            # Fallback sin zona (solo si tu runtime no tiene zoneinfo)
            today_ar = datetime.now().strftime("%d-%m-%Y")

        # TO (Graph) + SUBJECT fijo
        to_list  = [my_email] if my_email else []
        draft["to"] = to_list
        draft["subject"] = f"Reporte semanal — {today_ar}"

        
        # CC/BCC desde .env (dedup vs TO)
        cc_list, bcc_list = get_env_cc_bcc()
        to_lwr = {e.lower() for e in to_list}

        cc_final  = [e for e in cc_list  if e.lower() not in to_lwr]
        bcc_final = [e for e in bcc_list if e.lower() not in to_lwr]

        if cc_final:  draft["cc"]  = cc_final
        if bcc_final: draft["bcc"] = bcc_final

        # Normalizar links
        draft["body_html"] = urls_to_anchor_html(draft.get("body_html"))
        draft["body_text"] = urls_to_label_text(draft.get("body_text"))

        # Insertar banner real arriba del body_html (y quitar placeholder si vino del modelo)
        placeholder = '<div style="width:100%;height:120px;background:#0F172A;border-radius:8px;margin:0 0 16px 0;"></div>'
        if "body_html" in draft and draft["body_html"]:
            draft["body_html"] = draft["body_html"].replace(placeholder, "", 1)

        banner_html = _banner_block()
        if banner_html:
            draft["body_html"] = banner_html + (draft.get("body_html") or "")



        # Re-serializar la respuesta ya corregida
        text = json.dumps(data, ensure_ascii=False)

    except Exception as e:
        if debug:
            raise
        # devolvemos el texto crudo y la advertencia
        return "", {
            "error": f"No pude post-procesar  JSOON de salida: {e}",
            "model": model,
            "raw_excerpt": text_for_json[:600]
        }

    # 6) Guardado opcional
    saved_path = None
    if out:
        saved_path = os.path.abspath(out)
        with open(out, "w", encoding="utf-8") as f:
            f.write(text)

    # 7) Métricas de uso
    usage = {}
    if hasattr(response, "usage_metadata") and response.usage_metadata:
        u = response.usage_metadata
        usage = {
            "prompt_tokens": getattr(u, "prompt_token_count", None),
            "response_tokens": getattr(u, "candidates_token_count", None),
            "thoughts_tokens": getattr(u, "thoughts_token_count", None),
            "total_tokens": getattr(u, "total_token_count", None),
        }

    meta = {
        "model": model,
        "saved_to": saved_path,
        "usage": usage,
        "rows_in": len(rows),
        "rows_after_clamp": len(rows_compact),
        "system_tokens_est": t_system_only,
        "emails_tokens_est": t_emails_est,
        "input_total_tokens_est": t_input_total,

    }
    return text, meta

# =========================
# Invocación
# =========================
texto, meta = run_analysis(CONFIG)

if not texto or not texto.strip():
    print("No se obtuvo JSON decodificable. \nDetalles meta:", meta)
else:
    try:
        data = json.loads(texto)
        draft = data["email_draft"]
        print(f"✅ Informe generado correctamente")
        #print("Subject:", draft.get("subject", ""))
        #print("To:", ", ".join(draft.get("to", [])))
        #print("CC:", ", ".join(draft.get("cc", [])))
        #print("BCC:", ", ".join(draft.get("bcc", [])))
        #print(texto)
    except json.JSONDecodeError as e:
        print("JSONDecodeError al parsear la salida.\nMeta:", meta)
        #print("\nTexto(primeros 600 chars):", (texto or "")[:600])
        raise

✅ Informe generado correctamente


In [29]:
# ===== Guardar HTML Preview =====
import json, html, os
from pathlib import Path

model_output_str = texto

# 2) Parseo y obtención de campos
data = json.loads(model_output_str)
draft = data.get("email_draft", {}) if isinstance(data, dict) else {}
subject = (draft.get("subject") or "Reporte semanal").strip()
body_html = (draft.get("body_html") or "").strip()
body_text = (draft.get("body_text") or "").strip()

# 3) Fallback: si no hay body_html, convertir body_text a HTML básico
if not body_html and body_text:
    body_html = "<p>" + html.escape(body_text).replace("\n\n", "</p><p>").replace("\n", "<br>") + "</p>"
if not body_html:
    body_html = "<p>(Sin contenido)</p>"

# 4) Envoltorio HTML simple para email (opcional, se ve prolijo en navegadores)
WRAP = f"""<!doctype html>
<html lang="es">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1">
<title>{html.escape(subject)}</title>
<style>
  :root {{ --bg:#f4f5f7; --card:#ffffff; --text:#111827; --muted:#6b7280; --border:#e5e7eb; }}
  body {{ margin:0; background:var(--bg); color:var(--text); font:16px -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, Arial, "Noto Sans", "Helvetica Neue", sans-serif; }}
  .email {{ max-width: 760px; margin: 32px auto; background: var(--card); border: 1px solid var(--border); border-radius: 12px; box-shadow: 0 4px 16px rgba(0,0,0,0.06); overflow: hidden; }}
  .header {{ padding: 18px 22px; font-weight: 600; border-bottom: 1px solid var(--border); }}
  .content {{ padding: 18px 22px; line-height: 1.55; }}
  .content p {{ margin: 0 0 12px; }}
  .content ul {{ margin: 0 0 12px 22px; }}
  .content li {{ margin: 6px 0; }}
  a {{ color: #2563eb; text-decoration: none; }}
  a:hover {{ text-decoration: underline; }}
  .footer {{ padding: 16px 22px; color: var(--muted); border-top: 1px solid var(--border); font-size: 13px; }}
</style>
</head>
<body>
  <div class="email">
    <div class="header">{html.escape(subject)}</div>
    <div class="content">
      {body_html}
    </div>
    <div class="footer">Vista previa exportada · El cliente real de correo puede aplicar estilos distintos.</div>
  </div>
</body>
</html>"""

# 5) Guardar SOLO en disco (sin mostrar)
out_dir = Path(os.getenv("PREVIEW_DIR", "./previews"))
out_dir.mkdir(parents=True, exist_ok=True)  # crea ./previews si no existe
out_path = out_dir / "email_preview.html"
out_path.write_text(WRAP, encoding="utf-8")

print(f"Saved HTML preview in 'previews' folder: email_preview.html")

#print(f"Saved preview to: {out_path.resolve()}")
# Si querés abrirlo luego manualmente:
# import webbrowser; webbrowser.open(out_path.resolve().as_uri())


Saved HTML preview in 'previews' folder: email_preview.html


In [ ]:
# === Enviar el HTML por correo ===

# %pip install -q msal msal-extensions requests python-dotenv

import sys
from pathlib import Path
from dotenv import load_dotenv
import importlib, send_email_graph
importlib.reload(send_email_graph) #recargar el modulo jupyter

# Cargar .env (CLIENT_ID, TENANT_ID)
load_dotenv()

# Asegurate de que send_email_graph.py esté en la misma carpeta del notebook (o ajustá la ruta)
HELPER = Path("send_email_graph.py")
if not HELPER.exists():
    raise FileNotFoundError("No encuentro send_email_graph.py en esta carpeta.")

sys.path.append(str(HELPER.resolve().parent))
from send_email_graph import send_email_html

# draft viene de tu celda anterior:
# data = json.loads(texto); draft = data["email_draft"]
res = send_email_html(
    to=draft.get("to"),
    cc=draft.get("cc"),
    bcc=draft.get("bcc"),
    subject=draft.get("subject"),
    body_html=draft.get("body_html"),
    save_to_sent=True,
)

print("✅ Email enviado:", res)

✅ Email enviado: {'ok': True, 'to': ['rizzijp@agromargaritas.com'], 'cc': [], 'bcc': [], 'subject': 'Reporte semanal — 16-09-2025'}


## Resultados

La implementacion logra llegar a la solución esperada de generar un informe a partir de los emails leídos. Se usó un modelo de texto a imagen para generar el banner del informe. Este luego se sumó al output del modelo de texto a texto para completar el informe.

El resultado esperado se logró, aunque considero que sigue habiendo un margen de mejora, especialmente en cuanto al prompt para lograr un informe que sea lo mas adecuado para las necesidades de cada uno. Se intentó estandarizar el formato del informe para que el modelo no presente tanta variabilidad en sus outputs. Aunque no se logra un resultado 100% determinista, el informe sigue ciertos paramétros que la mayoría de las veces se cumplen dando un resultado más que aceptable. Este es el motivo por el que el prompt terminó siendo mucho mas extenso a lo planteado en un principio. Con mayor tiempo de prueba, en futuras versiones se podría buscar hacerlo mas eficiente y buscar mejores resultados con menor consumo de tokens. A pesar de que el consumo de tokens termino siendo elevado, principalmente por la cantidad de mails pero tambien por la extension del prompt, se pudieron hacer pequeños ajustes en el codigo para optimizar este consumo. 

En cuanto al modelo de imagen Flux.1 Schnell, quiero hacer la salvedad de que se usó únicamente por ser gratuito y para probar el funcionamiento de la implementación pero dista mucho de los resultados de los modelos más potentes que existen hoy en el mercado. La realidad es que no haría falta crear el banner cada vez que genera el informe. A los fines de las consignas del proyecto se realizó de esa manera, aunque no sería la más eficiente. Aclarado esto, que la generación del banner sea parte de la implementación permite que tanto el prompt como el modelo se puedan modificar a gusto del usuario.

## Conclusiones

Encaré este proyecto con la certeza de que era posible llevarlo a cabo pero sin contar con los suficientes conocimientos técnicos de programación. Gracias a la posibilidad de usar a la IA como un asistente de punta a punta, sumado a muchas horas de prueba y error, comienzo a experimentar las oportunidades que se empiezan a abrir con el uso de esta tecnología. Considero que aún con un resultado que tiene margen de mejora y con un codigo que se puede hacer mucho mas eficiente, el hecho de haber llegado al resultado esperado es más que satisfactorio.

Más allá de toda la parte de codigo que requirió mucho ida y vuelta para lograr el objetivo, quiero destacar la importancia del prompt ya que inicié el proyecto con uno muy básico y lo fui complejizando a medida que iba viendo los resultados. Dado que los modelos no son deterministas, en la medida que se sigan haciendo pruebas con diferentes emails o casillas de correos, el prompt se debería seguir puliendo. Asimismo, según si es casilla laboral o personal, el rubro en el cual uno trabaje y demás características, se podría customizar para cada caso en particular. En la medida que más se pueda ir adaptando a las necesidades de cada uno, el resultado indefectiblemente va a ir mejorando. Sin embargo, creo que como una primera aproximación al uso de este tipo de modelos, el objetivo está logrado y sirve de base para ser testeado y mejorado en el futuro. 